<a href="https://colab.research.google.com/github/yangjik/yang.study/blob/main/10_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D%EA%B3%BC_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

기존에는 train, test 이렇게 두번 데이터셋을 나눴는데 테스트 데이터를 사용하지 않고 과대적합인지 과소적합인지 확인하는 방법은 train 데이터 셋을 한번더 나눈 검증세트를 만드는 것이다.

In [ ]:
# 웹사이트 데이터 가져오기
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [ ]:
# csv파일에서 필요한 열 가져오기
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [ ]:
# 데이터 분리하기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, train_size=0.2, random_state=42)


In [ ]:
# 훈련세트를 이용한 검증세트 분배!!
# train : sub = 8 : 2
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [ ]:
sub_input.shape, val_input.shape

((1039, 3), (260, 3))

In [ ]:
# 결정트리 모델
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print('train 8 : ', dt.score(sub_input, sub_target), '\nval(검증세트) 2 : ', dt.score(val_input, val_target))


train 8 :  1.0 
val(검증세트) 2 :  0.8423076923076923


위 결과를 확인하면 과대적합 발생!!!

<h2>교차 검증</h2>

3-폴드 교차검증<br>

- 훈련 세트를 세 부분으로 나눠서 교차 검증를 수행하는 것.
- 훈련세트를 몇 부분으로 나누냐에 따라 다르게 불린다.


In [ ]:
# 교차 검증 모델
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
scores

{'fit_time': array([0.00285935, 0.00256348, 0.00238514, 0.00225902, 0.00231194]),
 'score_time': array([0.00085402, 0.00076079, 0.00063705, 0.00065374, 0.00064325]),
 'test_score': array([0.84615385, 0.81538462, 0.83076923, 0.82692308, 0.81853282])}

n-폴드 교차검증에서 cross_validate() 에서 cv 매개변수로 n을 지정할수있다.

In [ ]:
import numpy as np

np.mean(scores['test_score'])

0.8275527175527175

현재까지는 from sklearn.model_selection import train_test_split 통해서 데이터를 섞고 train, test 데이터를 분배했지만 기본적으로 회귀 모델 KFold 분할기를 사용하고 골고루 데이터를 섞기 위해 StratifiedKFold 를 이용한다.

In [ ]:
# KFold 분할기를 사용하기전 데이터 섞기
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())

np.mean(scores['test_score'])

0.8275527175527175

In [ ]:
# 위에서 데이터를 섞고 10-폴드 교차 검증 수행
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_validate(dt, train_input, train_target, cv=splitter)
np.mean(scores['test_score'])

0.8314370900417412

모델 파라미터 : 머신러닝 모델이 학습하는 파라미터<br>
하이퍼 파라미터 : 모델이 학습할 수 없어서 사용자가 지정해야만 하는 파라미터<br>

따라서 사이킷런과 같은 머신러닝 라이브러리를 사용할 때 이런 하이퍼 파라미터는 모두 클래스나 메서드의 매개변수로 표현한다.

## 하이퍼 파라미터 튜닝

- 먼저 라이브러리가 제공하는 기본값을 그대로 사용해서 모델 학습진행
- 검증 세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 바꿔본다.
- 결론 : 매개변수를 바꿔가면서 모델을 학습한 결과를 이용해서 최적의 하이퍼 파라미터를 찾는다.(plot이용해서 확인)

## 그리드 서치

- 하이퍼 파라미터 탐색과 교차 검증을 한 번에 수행
- cross_validate() 함수를 호출할 필요가 없다. -> n-폴드 교차검증

In [ ]:
from sklearn.model_selection import GridSearchCV

# 탐색할 매개변수와 탐색할 값의 리스트를 딕셔너리로 만들기
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [ ]:
# 탐색 대상모델과 params 변수를 전달하여 그리드 서치 객체 만들기
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)   # n_jobs=1  <- default = 1, 이 파라미터의 의미는 병렬 실행에 사용할 cpu 코어수를 지정 -1로 지정하면 시스템에 있는 모든 코어를 사용함.
                                                                                # cv=5      <- default = 5, n-폴드 교차검증

In [ ]:
# 그리드 서치 객체는 결정 트리모델 min_impurity_decrease 값을 바꿔가며 5번 실행

gs.fit(train_input, train_target)

# 위 학습은 5 x 5 = 25 의 모델 학습진행한다.

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
# 검증 점수가 가장 높은 모델의 매개변수 조합으로 전체 훈련 세트에서 자동으로 다시 모델을 훈련한다.
dt = gs.best_estimator_
dt.score(train_input, train_target)

0.9584295612009238

In [ ]:
# bets_params_ 로 가장 좋은 파라미터로 확인되었다.
print(gs.best_params_)

{'min_impurity_decrease': 0.0005}


In [ ]:
# 각 매개변수에서 수행한 교차 검증의 평균 점수 확인
gs.cv_results_['mean_fit_time']

array([0.00393901, 0.00532479, 0.00334177, 0.003334  , 0.00392509])

In [ ]:
#
best_index = np.argmax(gs.cv_results_['mean_test_score'])  # numpy에서 argmax() 함수 사용해서 가장 큰 값의 인덱스 추출
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0005}


위 과정 정리
- 먼저 탐색할 매개변수를 지정
- 그 다음 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.
- 그리드 서치는 최상의 매개변수에서(교차 검증에 사용한 훈련 세트가 아니라) 전체 훈련 세트를 사용해 최종 모델을 훈련. 이 모델도 그리드 서치 객체에 저장.

In [ ]:
# 결정 트리를 통해서 더 복잡한 매개변수 조합을 탐색
params = {
    'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
    'max_depth' : range(5, 20, 1),
    'min_samples_split' : range(2, 100, 10)
}

In [ ]:
#
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# 최상의 매개변수 조합 확인
print(gs.best_params_)

{'max_depth': 5, 'min_impurity_decrease': 0.0009000000000000001, 'min_samples_split': 22}


In [ ]:
# 최상의 교차 검증 점수 확인
np.max(gs.cv_results_['mean_test_score'])

0.866052866052866

## 랜덤 서치
- 매개변수 조건이 많아서 그리드 서치 수행 시간이 오래걸릴 때 랜덤 서치를 사용한다.
- 매개변수 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포객체를 전달.

In [ ]:
from scipy.stats import uniform, randint

In [ ]:
rgen = randint(0, 10) # 0 ~ 10 사이 랜덤 정수값
rgen.rvs(10)

array([1, 9, 3, 6, 7, 0, 3, 0, 1, 6])

In [ ]:
# 1000개를 샘플링 해서 각 숫자의 개수 카운트
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 86, 120,  97, 101,  95,  95, 100, 120,  98,  88]))

In [ ]:
# uniform 클래스도 위랑 동일. 0~1 사이의 10개의 실수를 추출
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.77989858, 0.48710393, 0.55538926, 0.94500949, 0.57165   ,
       0.47182575, 0.66603391, 0.00865424, 0.01531692, 0.70283578])

In [ ]:
# 랜덤 서치에 randint, uniform 클래스 객체를 넘겨주고 총 몇 번을 샘플링해서 최적의 매개변수 찾기
# min_samples_leaf 매개변수 탐색 대상에 추가
params = {
    'min_impurity_decrease' : uniform(0.001, 0.001),
    'max_depth' : randint(20, 50),
    'min_samples_split' : randint(2, 25),
    'min_samples_leaf' : randint(1, 25)
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
dtc = DecisionTreeClassifier(random_state=42)
gs = RandomizedSearchCV(dtc, params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cb52077ead0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7cb52077f250>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cb52077cd30>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7cb52077c910>},
                   random_state=42)

In [ ]:
# 위에서 학습한 최적의 파라미터 확인
gs.best_params_

{'max_depth': 48,
 'min_impurity_decrease': 0.0018583588048137198,
 'min_samples_leaf': 4,
 'min_samples_split': 20}

In [ ]:
# 교차 검증
np.max(gs.cv_results_['mean_test_score'])

0.8622007722007723

In [ ]:
# 테스트 데이터 성능확인
dt = gs.best_estimator_  # 위에서 학습시킨 모델
dt.score(test_input, test_target)

0.83801462100808